[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17GYI0VlTaoN_Ksjk_8B3237rX8F5OkDl?usp=sharing)

In [1]:
# érdemes előre telepíteni, és utána újraindítani
!pip3 install hyperas
!pip3 install hyperopt

In [2]:
import numpy as np  

In [3]:
"""# Plot some example images
import matplotlib.pyplot as plt

cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print('Example training images and their labels: ' + str([x[0] for x in y_train[0:5]])) 
print('Corresponding classes for the labels: ' + str([cifar_classes[x[0]] for x in y_train[0:5]]))

f, axarr = plt.subplots(1, 5)
f.set_size_inches(16, 4)

for i in range(5):
    img = X_train[i]
    axarr[i].imshow(img)
plt.show()"""

"# Plot some example images\nimport matplotlib.pyplot as plt\n\ncifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']\nprint('Example training images and their labels: ' + str([x[0] for x in y_train[0:5]])) \nprint('Corresponding classes for the labels: ' + str([cifar_classes[x[0]] for x in y_train[0:5]]))\n\nf, axarr = plt.subplots(1, 5)\nf.set_size_inches(16, 4)\n\nfor i in range(5):\n    img = X_train[i]\n    axarr[i].imshow(img)\nplt.show()"

In [4]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical 

def data():
  # Import the CIRAF10 dataset
  (X_train, y_train), (X_test, y_test) = cifar10.load_data()

  # Transform label indices to one-hot encoded vectors
  y_train = to_categorical(y_train, num_classes=10)
  y_test = to_categorical(y_test, num_classes=10)

  # Change data type
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  # Normalization of pixel values (to [0-1] range)
  X_train /= 255
  X_test /= 255
  
  return x_train, y_train, x_test, y_test

In [9]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping

import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def create_model(x_train, y_train, x_test, y_test):
  # hyperparam space:
  layer_size_1 = {{choice([16, 32, 64, 128, 256])}}
  layer_size_2 = {{choice([16, 32, 64, 128, 256])}}
  layer_size_3 = {{choice([16, 32, 64, 128, 256])}}
  dropout_1 = {{uniform(0, 0.5)}}
  dropout_2 = {{uniform(0, 0.5)}}
  dropout_3 = {{uniform(0, 0.5)}}
  activation = {{choice(['relu', 'leakyrelu', 'swish'])}}
  learning_rate = {{uniform(0.0001, 0.001)}}
  batch_size = {{choice([32, 128, 512])}}
  print('hyperparams: ', layer_size_1, layer_size_2, layer_size_3, dropout_1, dropout_2, dropout_3, activation, batch_size)

  # Crate model
  model = Sequential()

  model.add(Conv2D(layer_size_1, kernel_size=(3, 3), activation=activation, input_shape=(32,32,3,))) 
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( dropout_1))
  model.add(Conv2D(layer_size_2, kernel_size=(3, 3), activation=activation))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( dropout_2))
  model.add(Conv2D(layer_size_3, kernel_size=(3, 3), activation=activation))
  model.add(Dropout( dropout_3))

  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0006),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  # Train model
  result = model.fit(X_train, y_train, epochs=200, batch_size=32,
                    callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, verbose=0)],
                    validation_split=0.2, shuffle=True)
  # save best val acc
  best_val_acc = np.amax(result.history['val_accuracy']) 
  print('best val_acc:', best_val_acc)
  
  # Print logs and hyperparams into csv
  with open('hyperas-ciraf10-log.csv', 'a') as csv_file:
      csv_file.write(str(layer_size_1) + ';')
      csv_file.write(str(layer_size_2) + ';')
      csv_file.write(str(layer_size_2) + ';')
      csv_file.write(str(dropout_1) + ';')
      csv_file.write(str(dropout_2) + ';')
      csv_file.write(str(dropout_3) + ';')
      csv_file.write(str(activation) + ';')
      csv_file.write(str(learning_rate) + ';')
      csv_file.write(str(batch_size) + ';')
      csv_file.write(str(best_val_acc) + '\n')

  return {'loss': -best_val_acc, 'status': STATUS_OK, 'model': model}

In [10]:
# log fájl inicializálás / fejléc
with open('hyperas-ciraf10-log.csv', 'w') as csv_file:
  csv_file.write('layer_size_1' + ';')
  csv_file.write('layer_size_2' + ';')
  csv_file.write('layer_size_2' + ';')
  csv_file.write('dropout_1' + ';')
  csv_file.write('dropout_2' + ';')
  csv_file.write('dropout_3' + ';')
  csv_file.write('activation' + ';')
  csv_file.write('learning_rate' + ';')
  csv_file.write('batch_size' + ';')
  csv_file.write('best_val_acc' + '\n')

In [11]:
# ezután colab-ról le kell tölteni a notebook-ot
# és a .ipynb fájlt utána visszatölteni az aktuális könyvtárba

# erre azért van szükség, mert a hyperas a fenti kódben regexp-ekkel cseréli ki
# a hiperparaméter változókat az adott értékekre

# ha nem jupyter notebook-ban használjuk a hyperas-t, akkor nem kell ilyen trükközés

In [14]:
# teljes hiperparaméter optimalizálás indítása
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          notebook_name='DL_kishf5',
                                          trials=Trials())

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    from keras.datasets import cifar10
except:
    pass

try:
    from keras.utils.np_utils import to_categorical
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    import hyperas
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'layer_size_1': hp.choice('layer_size_1', [16, 32, 64, 128, 256]),
        'layer_size_1_1': hp.choice('layer_size_1_1', [16, 32, 64, 128, 256]),
        'layer_size_1_2': hp.choice('layer_size_1_2', [1

AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
print(model.metrics_names)
print(score)

['loss', 'accuracy']
[0.8043055534362793, 0.734000027179718]


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 6, 6, 64)         